In [1]:
import keras
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

#Importing the VAE and RNN.
import os
import sys
#Adding WorldModels path to pythonpath
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
#To access gym
import sys
sys.path.append('/home/kaiolae/.virtualenvs/worldmodels/lib/python3.5/site-packages')
import os

Using TensorFlow backend.


/home/kaiolae/code/word_models_keras_test/WorldModels


In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

def plot_movie_mp4(image_array):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [3]:
huge_data_path = "/mnt/data/doom_data_large/rnn_format_data/rnn_training_data.npz"
from batch_generator import KerasBatchGenerator

In [4]:
from VAE.world_model_vae import VAE
loading_model_path = "../vae_from_large_dataset/final_full_vae_weights.h5"
vae=VAE()
vae.set_weights(loading_model_path)
def decode_and_visualize(latent_vector_sequence, vae):
    plot_movie_mp4(vae.generate_picture_from_latent(latent_vector_sequence))

VAE init
Shape before flattening: (None, 16, 16, 32)
KL Shape: (None,)
Xent shape: ()
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 3)    39          encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 32)   416         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 16, 16, 32)   4128        conv2d_2[0][0]                   
_______________________

In [5]:
import numpy as np
#Loading z-values and actions. Expecting a compressed npz-file
rnn_training_data = np.load(huge_data_path)
action_data = rnn_training_data['action']
observation_data = rnn_training_data['latent']

In [6]:
VAL_SPLIT=0.15
sequence_length=100
BATCH_SIZE = 512
skip_ahead=5
validation_data_start_index = int((1-VAL_SPLIT)*len(observation_data))
train_data_generator = KerasBatchGenerator(observation_data[:validation_data_start_index], action_data[:validation_data_start_index], sequence_length, BATCH_SIZE, skip_ahead=skip_ahead)
val_data_generator = KerasBatchGenerator(observation_data[validation_data_start_index:], action_data[validation_data_start_index:], sequence_length, BATCH_SIZE, skip_ahead=skip_ahead)


In [7]:
training_generator = train_data_generator.generate()

In [8]:
x, y = next(training_generator)
x=np.array(x)
actions=x[:,:,-1] #Removing the action.
x=x[:,:,:-1] #Removing the action.
y=np.array(y)
print(x.shape)
print(actions[0])

(512, 100, 64)
[ 0.57335186  0.61330076  0.61330076  0.53736549  0.53736549 -0.32109988
 -0.32109988 -0.32109988 -0.32109988 -0.1735572  -0.1735572  -0.1735572
 -0.1735572  -0.94038147 -0.94038147  0.16199381  0.16199381 -0.60567428
 -0.60567428  0.48537656  0.48537656  0.48537656  0.48537656  0.48537656
  0.48537656 -0.50188925 -0.50188925 -0.85178578 -0.48363017  0.35714002
  0.35714002  0.35714002  0.35714002  0.54902701  0.45560918  0.45560918
  0.45560918  0.45560918  0.45560918  0.97835685 -0.3339568  -0.3339568
 -0.3339568  -0.3339568  -0.3339568  -0.3339568  -0.68271963 -0.68271963
 -0.68271963 -0.6043504  -0.6043504  -0.6043504  -0.6043504  -0.6043504
 -0.6043504  -0.29129042 -0.29129042 -0.53116677 -0.53116677 -0.53116677
 -0.53116677 -0.53116677 -0.53116677 -0.53116677 -0.53116677  0.75379844
  0.75379844  0.75379844  0.75379844  0.37934227  0.43251007  0.43251007
  0.43251007 -0.81144442 -0.81144442 -0.81144442 -0.81144442 -0.36420658
 -0.36420658  0.62927475  0.93032264 -0

In [10]:
decode_and_visualize(x[8], vae)

<Figure size 64x64 with 0 Axes>

In [ ]:
normal_data=observation_data[0]
decode_and_visualize(normal_data, rnn_analyzer=rnn_analyzer)